## Imports

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import math

from joblib import dump, load

lookup_df = pd.read_csv("zone_lookup.csv")

DATEPARSE = "%Y-%m-%d %H"

In [17]:
def save_model(model, poly, min_max):
    file_name = datetime.now().strftime(DATEPARSE)
    dump(model, "models/" + file_name + "_model.joblib")
    dump(poly, "models/" + file_name + "_poly.joblib")
    dump(min_max, "models/" + file_name + "_min_max.joblib")

# Train

In [19]:
#dfGreen = pd.read_csv("taxi_data/green_tripdata_2019_concat.csv", chunksize=50000)
dfYellow = pd.read_csv("taxi_data/yellow_tripdata_2019_concat.csv", chunksize=50000)

model = SGDRegressor()

poly = PolynomialFeatures(degree=12, include_bias=False)
min_max = MinMaxScaler()

counter = 0
best_accuracy = 0

for chunk in dfYellow:

    X_green = chunk[["PU_start", "PUlat", "PUlong","DOlat", "DOlong"]] # input cols
    y_green = chunk[["travel_time"]] # output cols
    
    scaled = poly.fit_transform(min_max.fit_transform(X_green.values))

    X_train, X_test, y_train, y_test = train_test_split(scaled, y_green.values.ravel(), test_size = 0.25)

    model.partial_fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    score = r2_score(y_test, y_pred)

    print(str(counter) + ": Mean squared error: " + str(mean_squared_error(y_test, y_pred)) + " Accuracy: " + str(score))
    if score > best_accuracy:
        print("New best accuracy")
        save_model(model, poly, min_max)
        best_accuracy = score
    counter += 1

0: Mean squared error: 85.08678029772538 Accuracy: -0.0021003361228904005
1: Mean squared error: 63.36800030572017 Accuracy: 0.10429701608099085
New best accuracy
2: Mean squared error: 69.00441723840346 Accuracy: 0.21764257890029537
New best accuracy
3: Mean squared error: 82.85062113439864 Accuracy: 0.2403039415849636
New best accuracy
4: Mean squared error: 86.04618557000849 Accuracy: 0.12396731886135226


KeyboardInterrupt: 